In [5]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/filtered_data.csv')

/tmp/ipykernel_26782/936502539.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/filtered_data.csv')


### Chat Bot Evaluation Questions

to evaluate the performance of the chat bot, we will ask a number of questions and see if the chat bot can answer them correctly. The correct answers will be provided by us.

In [38]:
# List Cancer related indicators available in the dataset.

answer_1 = df[df.Topic == 'Cancer']['Question'].unique()
answer_1

array(['Cancer of the oral cavity and pharynx, mortality',
       'Cancer of the prostate, mortality',
       'Invasive cancer (all sites combined), mortality',
       'Cancer of the female breast, mortality',
       'Cancer of the female cervix, mortality',
       'Cancer of the colon and rectum (colorectal), mortality',
       'Cancer of the lung and bronchus, mortality',
       'Invasive melanoma, incidence', 'Melanoma, mortality',
       'Invasive cancer of the oral cavity or pharynx, incidence',
       'Invasive cancer of the prostate, incidence',
       'Invasive cancer (all sites combined), incidence',
       'Invasive cancer of the female breast, incidence',
       'Invasive cancer of the cervix, incidence',
       'Cancer of the colon and rectum (colorectal), incidence',
       'Cancer of the lung and bronchus, incidence',
       'Mammography use among women aged 50-74 years',
       'Fecal occult blood test, sigmoidoscopy, or colonoscopy among adults aged 50-75 years',
      

In [33]:
# List Cancer related Indicators for which there are data for the year the last year of any available data.
answer_2 =  len(df[(df.Topic == 'Cancer') & (df.YearStart == df.YearStart.max())])
answer_2

0

In [35]:
# How many diferent datatypes are in the dataset?
answer_3 = len(df['DataValueType'].unique())
answer_3

12

In [42]:
# What is the mean value for indicator 'Fecal occult blood test, sigmoidoscopy, or colonoscopy among adults aged 50-75 years' in the last year of available data? Provide answer for Overall stratification group.

max_year = df[df.Question == 'Fecal occult blood test, sigmoidoscopy, or colonoscopy among adults aged 50-75 years']['YearStart'].max()

answer_4 = df[(df.Question == 'Fecal occult blood test, sigmoidoscopy, or colonoscopy among adults aged 50-75 years') & (df.YearStart == max_year) & (df.Stratification1 == "Overall")]['DataValue'].mean()
answer_4

73.12222222222223

In [66]:
# What is the value for 'Cancer of the female breast, mortality' in Wisconsin. Provide answer for Overall stratification group.

answer_5 = df[(df.Question == 'Cancer of the female breast, mortality') & (df.Stratification1 == 'Overall') & (df.LocationAbbr == 'WI')][['YearStart', 'YearEnd', 'DataValue']].sort_values(by='YearStart', ascending=False)
answer_5

,YearStart,YearEnd,DataValue
18180,2015,2019,720.0
17772,2014,2018,729.0
18992,2013,2017,726.0
18543,2012,2016,743.0
18720,2011,2015,753.0
17331,2010,2014,763.0
19238,2009,2013,767.0
18357,2008,2012,758.0


In [126]:
# Which state has the worst values for 'Cancer of the female breast, mortality' for the last year of available data for that particular indictaor. Where does it rank in Poverty inicator among states? Provide answer for Overall stratification group.


cancer_mortality = df[(df['Question'] == 'Cancer of the female breast, mortality') &
                      (df['Stratification1'] == 'Overall') &
                      (df['LocationAbbr'] != 'US')]

cancer_mortality.DataValue = cancer_mortality.DataValue.astype(float)

max_year = cancer_mortality['YearEnd'].max()
worst_state_data = cancer_mortality[cancer_mortality['YearEnd'] == max_year].nlargest(1, 'DataValue')
worst_state = worst_state_data['LocationDesc'].iloc[0] if not worst_state_data.empty else None

# If a worst state is found, proceed to find its poverty ranking
if worst_state:
    poverty_data = df[(df['Question'] == 'Poverty') & 
                      (df['Stratification1'] == 'Overall') & 
                      (df['YearEnd'] == max_year)]
    poverty_data['Rank'] = poverty_data['DataValue'].rank(ascending=True)
    worst_state_poverty_rank = poverty_data[poverty_data['LocationDesc'] == worst_state]['Rank'].iloc[0] if not poverty_data.empty else None

    answer_6 = (worst_state, worst_state_poverty_rank)
else:
    answer_6 = ("No state found for the given criteria", None)

answer_6


/tmp/ipykernel_26782/3642622698.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cancer_mortality.DataValue = cancer_mortality.DataValue.astype(float)
/tmp/ipykernel_26782/3642622698.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poverty_data['Rank'] = poverty_data['DataValue'].rank(ascending=True)


('California', 15.0)

In [127]:
answer_6

('California', 15.0)

In [86]:
# How does 'Hospitalizations for Asthma' indicator corelate with 'Sale of cigarette packs' Provide answer for Overall stratification group.


astma_hospitalizations = df[(df['Question'] == 'Hospitalizations for asthma') &
                            (df['Stratification1'] == 'Overall') &
                            (df['LocationAbbr'] == 'US')][['YearStart','DataValue']]

astma_hospitalizations.DataValue = astma_hospitalizations.DataValue.astype(float)

cigarete_sales = df[(df['Question'] == 'Sale of cigarette packs') &
                    (df['Stratification1'] == 'Overall') &
                    (df['LocationAbbr'] == 'US')][['YearStart','DataValue']]

cigarete_sales.DataValue = cigarete_sales.DataValue.astype(float)

correlation = astma_hospitalizations.DataValue.corr(cigarete_sales.DataValue)

answer_7 = correlation

answer_7

nan

In [98]:
import numpy as np
# Filter for United States data for 'Overall' stratification
df_us_overall = df[(df['LocationDesc'] == 'United States') & (df['Stratification1'] == 'Overall')]

# Pivot the DataFrame to have indicators as columns, each row represents a year
pivot_df = df_us_overall.pivot(index='YearEnd', columns='Question', values='DataValue')
pivot_df = pivot_df.apply(pd.to_numeric, errors='coerce')
# Compute the correlation matrix
correlation_matrix = pivot_df.corr()

# Find the pair with the highest correlation
# We use np.nan_to_num to replace NaNs with zero to avoid comparing NaNs
# We also replace 1s (perfect correlation of an indicator with itself) with 0 to avoid self-correlation
correlation_matrix = np.nan_to_num(correlation_matrix.to_numpy(), nan=0.0)
np.fill_diagonal(correlation_matrix, 0)

# Get the indices of the maximum correlated pair
max_corr_indices = np.unravel_index(np.argmax(correlation_matrix, axis=None), correlation_matrix.shape)
max_corr_value = correlation_matrix[max_corr_indices]

# Get the indicator names for the most correlated pair
most_correlated_pair = pivot_df.columns[list(max_corr_indices)]

# Output the names of the two most correlated indicators and their correlation coefficient
answer_8 = most_correlated_pair[0], most_correlated_pair[1], max_corr_value
answer_8

('Census tracts with healthier food retailers within � mile of boundary',
 'Overweight or obesity among adults aged >= 18 years',
 1.000000000000071)

In [109]:
# Which state has the highest number of missing values for 'Poverty' indicator? Provide answer for Overall stratification group.

poverty_data = df[(df['Question'] == 'Poverty') & (df['Stratification1'] == 'Overall')]
# Count the number of missing values (NaN) in the 'DataValue' column for each state
missing_values_count = poverty_data['DataValue'].isna().groupby(poverty_data['LocationDesc']).sum()

# Find the state with the highest number of missing values
state_with_most_missing_values = missing_values_count.idxmax()
number_of_missing_values = missing_values_count.max()

# Output the state with the most missing values and the count of missing values
answer_9 = (state_with_most_missing_values, number_of_missing_values)
print(answer_9)

# the answer should be : There are no missing values for this selection criteria

('Alabama', 0)


In [123]:
# Which state ranks the highest in poverty for startification group 'Hispanic' in the last year of available data?

df_poverty_hispanic = df[(df['Question'] == 'Poverty') & (df['Stratification1'] == 'Hispanic')]

# Find the latest year of available data for this group
latest_year = df_poverty_hispanic['YearEnd'].max()

# Filter the data for this latest year
df_latest_year_poverty_hispanic = df_poverty_hispanic[df_poverty_hispanic['YearEnd'] == latest_year]

df_latest_year_poverty_hispanic.DataValue = df_latest_year_poverty_hispanic.DataValue.astype(float)

# Rank the states based on the 'DataValue' which represents the poverty level, assuming that a higher value means higher poverty
# Sort the DataFrame based on 'DataValue' in descending order to have the state with the highest poverty on top
df_latest_year_poverty_hispanic_sorted = df_latest_year_poverty_hispanic.sort_values(by='DataValue', ascending=False)

# The state with the highest rank in poverty will be the first entry in the sorted DataFrame
state_highest_poverty_hispanic = df_latest_year_poverty_hispanic_sorted.iloc[0]['LocationDesc'] if not df_latest_year_poverty_hispanic_sorted.empty else None

answer_10 = state_highest_poverty_hispanic



/tmp/ipykernel_26782/57765857.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_latest_year_poverty_hispanic.DataValue = df_latest_year_poverty_hispanic.DataValue.astype(float)


In [128]:
for i in range(1, 11):
    print(f"Answer {i}: {globals()['answer_' + str(i)]}")







Answer 1: ['Cancer of the oral cavity and pharynx, mortality'
 'Cancer of the prostate, mortality'
 'Invasive cancer (all sites combined), mortality'
 'Cancer of the female breast, mortality'
 'Cancer of the female cervix, mortality'
 'Cancer of the colon and rectum (colorectal), mortality'
 'Cancer of the lung and bronchus, mortality'
 'Invasive melanoma, incidence' 'Melanoma, mortality'
 'Invasive cancer of the oral cavity or pharynx, incidence'
 'Invasive cancer of the prostate, incidence'
 'Invasive cancer (all sites combined), incidence'
 'Invasive cancer of the female breast, incidence'
 'Invasive cancer of the cervix, incidence'
 'Cancer of the colon and rectum (colorectal), incidence'
 'Cancer of the lung and bronchus, incidence'
 'Mammography use among women aged 50-74 years'
 'Fecal occult blood test, sigmoidoscopy, or colonoscopy among adults aged 50-75 years'
 'Recent Papanicolaou smear use among women aged 21-44 years'
 'Papanicolaou smear use among adult women aged 21-65 